In [3]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

model_type = ModelType.mistral_7b_instruct_v2
llm_engine = get_vllm_engine(
    model_type, 
    model_id_or_path = '/home/qiuyang/workplace/swift/examples/pytorch/llm/output/mistral-7b-chat-v2/v10-20240120-151243/checkpoint-1948-merged',
    tensor_parallel_size=2,
    seed=42
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)


In [2]:
generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)

content = '''"Ukranian Prime Min Viktor F Yanukovich visits Washington for low key meetings with Vice Pres Dick Cheney and Sec of State Condoleezza Rice; says Ukraine is in no hurry to join NATO or European Union (S)"'''
eval_prompt = f"### Instruction: According to the following information:<{content}> Answer the question. Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination,year-month],just country name.\n### Response:\n"

request_list = [
    # {'query': "Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.\n\nCLAIM:\nPublication date: 2020-08-13\nContent: Illinois Strengthens Face Mask Rules in Businesses\n\nINFORMATION:\nInformation 1:\nPublication date: None\nTitle: FAQ for Businesses Concerning Use of Face-Coverings During COVID-19\nContent:\nA: Businesses reserve the right to refuse service to persons unable to comply with the requirement to wear a face covering, but they are required to provide a reasonable accommodation if it does not cause an undue hardship. Businesses are encouraged to inform customers there are exceptions to the requirement that all individuals must wear a mask.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. A: A face covering is a mask or cloth face covering that covers your nose and mouth.\nExceptions may be made for individuals with medical conditions or disabilities that prevent them from safely wearing a face covering. For more information, refer to the questions on reasonable accommodations. A: Masks still must be worn by everyone on planes, buses, trains, and other forms of public transportation; in transportation hubs, such as airports and train and bus stations; in health care settings; and in congregate facilities, such as correctional facilities and homeless shelters.\nThese frequently asked questions are to provide guidance regarding the application of the face covering requirement in Executive Order 2021-10 for businesses and other places of public accommodation subject to Article 5 of the Illinois Human Rights Act, 775 ILCS 5/. When Face Coverings are Required Q: What does it mean to wear a face covering?\nInformation 2:\nPublication date: 2013-08-06\nTitle: Illinois Strengthens Face Mask Rules in Businesses - WebMD\nContent:\nCOVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.\nInformation 3:\nPublication date: 2020-09-02\nTitle: Illinois Businesses Face Fines For Failing To Require Masks | WBEZ ...\nContent:\nPritzker’s new rule, approved by a 12-member state board Tuesday, would allow retailers to be fined up to $2,500 if they don’t require masks to be worn. ... URL Copied! WBEZ brings you fact-based news and information. Sign up for our newsletters to stay up to date on the stories that matter. A state panel Tuesday sided with Illinois Gov. JB Pritzker in his fight against COVID-19 by approving a rule that could mean fines of up to $2,500 for “rogue” businesses that don’t require patrons wear facial coverings.\nNow, the only consequences would be felt by businesses and their financial bottom lines. But that change wasn’t enough to satisfy Republicans, who failed in their effort to block the new rule from taking effect. The GOP pushback fell two votes shy of the necessary eight votes required to block the rule. “Honestly, having talked with some of the Illinoisans who refuse to wear masks, passing this administrative rule is not going to make people more likely to wear a mask,” said state Sen.\nSince then, Pritzker’s administration eliminated a provision from the earlier version of the rule that would have subjected individuals who run businesses that flout state mask-wearing requirements to a Class A misdemeanor, which theoretically could carry jail time upon conviction.\nLocal public health departments or law-enforcement agencies would first have to give rule-breaking businesses or groups written notice that they were violating the state mask-wearing order. Then, if that step didn’t produce compliance, local officials could order people in the business or facility to disperse.\nInformation 4:\nPublication date: 2023-04-14\nTitle: Mask Mandate FAQ - Illinois Manufacturers' Association\nContent:\nWhile the mask mandate applies to all employers, these new fines and penalties are aimed at consumer-facing businesses that are open to the public. Q: What businesses may be fined under this new rule? A: This new rule applies to any business, service, facility, or organization · open to the public. In these buildings open to the public, individuals should wear face coverings when unable to maintain a 6-foot social distance.\nA: This new rule applies to any business, service, facility, or organization · open to the public. In these buildings open to the public, individuals should wear face coverings when unable to maintain a 6-foot social distance. Generally speaking, manufacturing facilities are not open to the public and will not be significantly impacted by these new fines and penalties. Businesses may also be subject to penalties for hosting gatherings of more than 50 individuals.\nIn these buildings open to the public, individuals should wear face coverings when unable to maintain a 6-foot social distance. Generally speaking, manufacturing facilities are not open to the public and will not be significantly impacted by these new fines and penalties. Businesses may also be subject to penalties for hosting gatherings of more than 50 individuals. The specific language in the Administrative Rule provides “any businesses, service, facility or organization\nThe specific language in the Administrative Rule provides “any businesses, service, facility or organization · open to the public or employees shall require employees, customers, or other individuals who are over the age of two and able to medically tolerate a face covering to cover their nose and mouth with a face covering when on premises and unable to maintain a 6-foot social distance.\nInformation 5:\nPublication date: 2021-05-18\nTitle: Pritzker eases Illinois face covering rules, supports those still ...\nContent:\nBut they still call for wearing masks in crowded indoor settings — such as buses, planes, hospitals, prisons and homeless shelters. Under Pritzker’s updated rules, Illinoisans — whether vaccinated or not — will still be required to wear masks on public transportation and in transportation hubs, in correctional facilities and homeless shelters and in healthcare settings. Children will be required to wear masks in schools — those in daycare will also still be required to wear a face covering, according to a news release announcing the state’s changes.\nBut with the updated masking guidance, and the state now in the “bridge” phase of Pritzker’s reopening plan, the end of the pandemic could be near for Illinois. Gov. J.B. Pritzker puts on a mask during a briefing at the James R. Thompson Center in the Loop in November. ... The new CDC guidance announced last week allowed for fully vaccinated people to safely stop wearing masks outdoors and in the majority of indoor settings. But they still call for wearing masks in crowded indoor settings — such as buses, planes, hospitals, prisons and homeless shelters. Under Pritzker’s updated rules, Illinoisans — whether vaccinated or not — will still be required to wear masks on public transportation and in transportation hubs, in correctional facilities and homeless shelters and in healthcare settings.\nPritzker to ease Illinois rules on face coverings for fully vaccinated following CDC update · Shot of J&J for JB — Pritzker gets COVID-19 vaccine: ‘I feel great’ · As of Monday, the state has administered 10.4 million doses of the vaccine. A little over 4.8 million people in the state are fully vaccinated, according to data from the department of public health. The state’s updated policy on masks follows changes from the U.S.\n“With public health experts now saying fully vaccinated people can safely remove their masks in most settings, I’m pleased to follow the science and align Illinois’ policies with the CDC’s guidance.” · Pritzker went on in his statement to say he supports individuals and business who choose to continue to wear a mask “out of an abundance of caution as this pandemic isn’t over yet.” · The Democratic governor is one of those still wearing a mask — nearly eight weeks after receiving a dose of the Johnson & Johnson vaccine in Springfield. Asked about his face covering at an unrelated news conference on Monday, Pritzker said he thinks all residents “are going to have to adjust to the idea” of not needing to wear a mask.\n"},
    {'query': eval_prompt},
]

resp_list = inference_vllm(
    llm_engine, template, request_list, generation_config=generation_config, 
    # verbose=True, prompt_prefix="", output_prefix=""
)

for request, resp in zip(request_list, resp_list):
    print(f"query: {request['query']}")
    print()
    print(f"response: {resp['response']}")


query: ### Instruction: According to the following information:<"Ukranian Prime Min Viktor F Yanukovich visits Washington for low key meetings with Vice Pres Dick Cheney and Sec of State Condoleezza Rice; says Ukraine is in no hurry to join NATO or European Union (S)"> Answer the question. Whether anyone in the information was transferred between countries in text?if so,from which country to which country?What year and month did the transfer take place?only give the starting point and destination example [Yes or No,starting point,destination,year-month],just country name.
### Response:


response: [Yes,Ukraine,the United States,2008-03]
